# Imports

In [1]:
import pandas as pd
import vectorbtpro as vbt
import numpy as np
from typing import Dict

# Initialization

In [2]:
vbt.settings.wrapping ["freq"]                = "1m"
vbt.settings.portfolio['init_cash']           = 10000

# Settings and Parameters

In [3]:
pickle_files_path = "../data/150_prediction/*.pkl"
prediction_window = 150

# Local Imports

In [4]:
import os
import sys

sys.path.append(os.getcwd())

In [5]:
from lstm_analysis_utils import (read_pickle_files_into_df, add_forward_prices_to_df, generate_fwd_actual_column
                                 , generate_df_with_euclidean_distances, calculate_slopes, calculate_correlation_slopes
                                 , store_backtest_results)
from lstm_analysis_constants import ActionType, BASELINE_BACKTEST_RESULT_KEY

# Processing

In [6]:
df = read_pickle_files_into_df(pickle_files_path)

In [7]:
add_forward_prices_to_df(df, prediction_window)
df = df.copy()  # for large prediction_window size, the copy() call eliminates the fragmented dataframe warning

/home/htram/onramp/sigma/lstm_backtest/lstm_analysis_utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = df['BTCUSDT_Open'].shift(-i)
/home/htram/onramp/sigma/lstm_backtest/lstm_analysis_utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = df['BTCUSDT_Open'].shift(-i)
/home/htram/onramp/sigma/lstm_backtest/lstm_analysis_utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [8]:
generate_fwd_actual_column(df)

In [9]:
df = generate_df_with_euclidean_distances(df, prediction_window)

In [10]:
calculate_slopes(df)

In [11]:
calculate_correlation_slopes(df)

Correlation between Euclidean distance between long array and short array and future actual results:  0.1618181510042358
Correlation between difference in long minus short predictions and future actual results for longs:  0.7522970688352852
Correlation between difference in long minus short predictions and future actual results for shorts:  0.7553218863809256
Correlation between long slopes and future results: -0.02729435481597804
Correlation between short slopes and future results: 0.056368180654982974


In [12]:
#new_df.to_csv('backtest_results.csv') # This file is most of what the team will need to perform analysis

# Running backtest

- ### Storing the results of the backtests

In [13]:
backtest_result_dict: Dict[str, Dict] = {}

- ### Baseline backtest - just listen to LSTM

In [14]:
entries         = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_LONG  ), True, False))
exits           = pd.Series(np.where((df['recommendations'] == ActionType.CLOSE_LONG ), True, False))
short_entries   = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_SHORT ), True, False))
short_exits     = pd.Series(np.where((df['recommendations'] == ActionType.CLOSE_SHORT), True, False))

pf = vbt.Portfolio.from_signals(
    high                = df['BTCUSDT_High'],
    low                 = df['BTCUSDT_Low'],
    open                = df['BTCUSDT_Open'],
    close               = df['BTCUSDT_Close'],
    entries             = entries, # commented out for a short only backtest
    exits               = exits,
    short_entries       = short_entries,
    short_exits         = short_exits,    
    time_delta_format   = 'Rows', # Use the row index to calculate the time delta    
    )


In [15]:
store_backtest_results(BASELINE_BACKTEST_RESULT_KEY, pf.stats(), backtest_result_dict)

--------baseline_backtest_result--------
Start                                  1641686580.00000
End                                    1672165200.00000
Period                                 30 days 08:50:00
Start Value                                 10000.00000
Min Value                                    9891.67726
Max Value                                   25839.09455
End Value                                   24580.65133
Total Return [%]                              145.80651
Benchmark Return [%]                          -59.99879
Total Time Exposure [%]                        99.24537
Max Gross Exposure [%]                        169.20058
Max Drawdown [%]                               27.65931
Max Drawdown Duration                   9 days 12:08:00
Total Orders                                         35
Total Fees Paid                                 0.00000
Total Trades                                         35
Win Rate [%]                                   67.64706
Best Tr

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


- ### Other backtests - work in progress and code may not execute at all!!!

In [16]:
entry_slope_threshold       = [x / 100.0 for x in range(0, 8)]
short_entry_slope_threshold = [x / 100.0 for x in range(0, 9)]

for t1 in entry_slope_threshold:
  for t2 in short_entry_slope_threshold:
    entries       = pd.Series(np.where((df['long_slope' ] > t1  ), True, False))
    short_entries = pd.Series(np.where((df['short_slope'] < -t2 ), True, False))

    num_trades = (entries == True).sum() + (short_entries == True).sum()

    if num_trades > 0:    
      pf = vbt.Portfolio.from_signals(
          high              = df['BTCUSDT_High'],
          low               = df['BTCUSDT_Low'],
          open              = df['BTCUSDT_Open'],
          close             = df['BTCUSDT_Close'],
          entries           = entries, # commented out for a short only backtest          
          short_entries     = short_entries,
          td_stop           = prediction_window, # Hold on to the position for 8 bars
          time_delta_format = 'Rows', # Use the row index to calculate the time delta
          # tp_stop = 0.01,
          accumulate        =False,
          # sl_stop = 0.005,
          )    
      
      key = f"entry_slope_threshold_{t1}_short_entry_slope_threshold_{t2}"
      store_backtest_results(key, pf.stats(), backtest_result_dict)      

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


--------entry_slope_threshold_0.0_short_entry_slope_threshold_0.0--------
Start                                  1641686580.00000
End                                    1672165200.00000
Period                                 30 days 08:50:00
Start Value                                 10000.00000
Min Value                                    5507.20372
Max Value                                   14170.03265
End Value                                   10641.55297
Total Return [%]                                6.41553
Benchmark Return [%]                          -59.99879
Total Time Exposure [%]                        98.58678
Max Gross Exposure [%]                        120.58658
Max Drawdown [%]                               46.61553
Max Drawdown Duration                  15 days 08:46:00
Total Orders                                       2293
Total Fees Paid                                 0.00000
Total Trades                                       2250
Win Rate [%]                  

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


--------entry_slope_threshold_0.0_short_entry_slope_threshold_0.02--------
Start                         1641686580.00000
End                           1672165200.00000
Period                        30 days 08:50:00
Start Value                        10000.00000
Min Value                           3474.76584
Max Value                          10540.60939
End Value                           3696.84521
Total Return [%]                     -63.03155
Benchmark Return [%]                 -59.99879
Total Time Exposure [%]               94.51406
Max Gross Exposure [%]               100.00000
Max Drawdown [%]                      67.03449
Max Drawdown Duration         29 days 23:27:00
Total Orders                               551
Total Fees Paid                        0.00000
Total Trades                               276
Win Rate [%]                          44.36364
Best Trade [%]                        10.61814
Worst Trade [%]                       -8.93745
Avg Winning Trade [%]           

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


...

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


--------entry_slope_threshold_0.0_short_entry_slope_threshold_0.08--------
Start                         1641686580.00000
End                           1672165200.00000
Period                        30 days 08:50:00
Start Value                        10000.00000
Min Value                           3474.76584
Max Value                          10540.60939
End Value                           3696.84521
Total Return [%]                     -63.03155
Benchmark Return [%]                 -59.99879
Total Time Exposure [%]               94.51406
Max Gross Exposure [%]               100.00000
Max Drawdown [%]                      67.03449
Max Drawdown Duration         29 days 23:27:00
Total Orders                               551
Total Fees Paid                        0.00000
Total Trades                               276
Win Rate [%]                          44.36364
Best Trade [%]                        10.61814
Worst Trade [%]                       -8.93745
Avg Winning Trade [%]           

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


--------entry_slope_threshold_0.03_short_entry_slope_threshold_0.0--------
Start                         1641686580.00000
End                           1672165200.00000
Period                        30 days 08:50:00
Start Value                        10000.00000
Min Value                           7627.83277
Max Value                          23179.01839
End Value                          20670.79869
Total Return [%]                     106.70799
Benchmark Return [%]                 -59.99879
Total Time Exposure [%]               94.50492
Max Gross Exposure [%]               134.79081
Max Drawdown [%]                      38.02857
Max Drawdown Duration         10 days 04:14:00
Total Orders                               551
Total Fees Paid                        0.00000
Total Trades                               276
Win Rate [%]                          54.90909
Best Trade [%]                         9.10798
Worst Trade [%]                      -12.71398
Avg Winning Trade [%]           

/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period
/home/htram/onramp/sigma/.venv/lib/python3.10/site-packages/vectorbtpro/base/wrapping.py:960: UserWarning: Index is neither datetime-like nor integer
  return self.index_acc.dt_period


In [ ]:
# # Set up entries and exits based on the slope
# entry_slope_threshold = 0.0000
# short_entry_slope_threshold = 0.0000
# #entries = pd.Series(np.where((new_df['long_slope'] > entry_slope_threshold), True, False))
# long_minus_short_t = 1.2
# entries = pd.Series(np.where((new_df['long_minus_short'] > long_minus_short_t), True, False))
# #short_entries = pd.Series(np.where((new_df['short_slope'] < -short_entry_slope_threshold), True, False))
# #short_entries = pd.Series(np.where((new_df['long_minus_short'] < long_minus_short_t), True, False))
# pf = vbt.Portfolio.from_signals(
#     high=new_df['BTCUSDT_High'],
#     low=new_df['BTCUSDT_Low'],
#     open=new_df['BTCUSDT_Open'],
#     close=new_df['BTCUSDT_Close'],
#     entries=entries, # commented out for a short only backtest
#     # exits = result_df['long_slope']<0.04,
#     #short_entries=short_entries,
#     td_stop = prediction_window, # Hold on to the position for 8 bars
#     time_delta_format = 'Rows', # Use the row index to calculate the time delta
#     # tp_stop = 0.01,
#     accumulate=False,
#     # sl_stop = 0.005,
#     )
# print(pf.stats())

In [ ]:
# print(f"Num entries = {(entries == True).sum()}")
# print(f"Num short entries = {(short_entries == True).sum()}")
# new_df["long_slope"].describe()